# 🧼 Data Preprocessing Pipeline

In this section, we use the preprocessing pipeline to preprocess the data. A sequence of steps is applied to the data to clean it and prepare it for further analysis.

In [54]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
import sys
import os

path_to_preprocessing = os.path.join('..', '..', 'src')
sys.path.insert(0, path_to_preprocessing)

from preprocessing.preprocessing_pipeline import preprocess_files

In [56]:
preprocess_files()

📁 Found 3 files in the data folder.
⏳ Preprocessing data file for day 2023-04-04...
	Translating columns...
	Processing product_id...
	Processing transport_type...
	Processing stops...
	Processing arrivals...
	Processing departures...
	Handling inconsistent rows...
	Extracting operator data...
	Deleting unnecessary columns...
	Saving data...
😀 Done!
⏳ Preprocessing data file for day 2023-04-05...
	Translating columns...
	Processing product_id...
	Processing transport_type...
	Processing stops...
	Processing arrivals...
	Processing departures...
	Handling inconsistent rows...
	Extracting operator data...
	Deleting unnecessary columns...
	Saving data...
😀 Done!
⏳ Preprocessing data file for day 2023-04-06...
	Translating columns...
	Processing product_id...
	Processing transport_type...
	Processing stops...
	Processing arrivals...
	Processing departures...
	Handling inconsistent rows...
	Extracting operator data...
	Deleting unnecessary columns...
	Saving data...
😀 Done!
